# BB84 Quantum Key Distribution

This notebook demonstrates the BB84 Quantum Key Distribution protocol from Chapter 5, Single-Qubit Quantum Gates.

## 0. Setup

The cells below install the packages and import the libraries needed by this demo.

In [ ]:
#@title Install Qiskit and other dependencies
%pip install qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [2]:
# Imports

from IPython.display import display, Markdown
from math import cos, sin, pi as π
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.circuit import ClassicalRegister, QuantumCircuit, QuantumRegister

## 1. Quantum Random Number Generator

The following cell defines a Quantum Random Number Generator (QRNG) circuit. This circuit generates a random bit by preparing a qubit in the state $|+\rangle$, then measuring it in the $Z$ basis.

*Note*: The sample circuits in this repository use the Qiskit AER simulator. 
As such, all quantum randomness is simulated, actually using a deterministic pseudo-random number generator.
However, these same circuits could in principle be run on a real quantum computer to generate true quantum randomness.

In [3]:
def qrng():
    """
    Generate a random bit using a quantum random number generator (QRNG).

    This function creates a quantum circuit with a single qubit, applies a Hadamard gate to put the qubit
    into a superposition state, and then measures the qubit. The result is a random bit (0 or 1) due to
    the inherent randomness of quantum measurement.

    Returns:
        int: A random bit (0 or 1) generated by the quantum circuit.
    """
    q = QuantumRegister(1)
    circuit = QuantumCircuit(q)
    circuit.h(q)
    circuit.measure_all()
    counts = AerSimulator().run(circuit).result().get_counts()
    return int(list(counts.keys())[0])


bits = [qrng() for _ in range(8)]
display(Markdown(f"Random bits: {bits}"))

Random bits: [0, 1, 0, 1, 1, 0, 1, 1]

In [4]:
def run(n_bits, e_bits, eavesdropper=False):
    alice_bases = []
    eve_bases = []
    bob_bases = []
    alice_results = []
    bob_results = []
    eve_results = []
    basis_matches = []
    for i in range(n_bits):
        q = [q0] = QuantumRegister(1, 'q')
        c_alice = ClassicalRegister(1, 'alice')
        c_eve = ClassicalRegister(1, 'eve')
        c_bob = ClassicalRegister(1, 'bob')
        circuit = QuantumCircuit(q, c_alice, c_eve, c_bob)
        if qrng():
            circuit.h(q0)

        alice_basis = 'DA' if qrng() else 'HV'
        alice_bases.append(alice_basis)
        if alice_basis == 'DA':
            circuit.h(q0)
        circuit.measure(q, c_alice)

        eve_basis = 'DA' if qrng() else 'HV'
        eve_bases.append(eve_basis)
        if eve_basis == 'DA':
            circuit.h(q0)
        circuit.measure(q, c_eve)    

        bob_basis = 'DA' if qrng() else 'HV'
        bob_bases.append(bob_basis)
        if bob_basis != alice_basis:
            circuit.h(q0)
        circuit.measure(q, c_bob)

        simulator = AerSimulator()
        sampler = Sampler(simulator)
        job = sampler.run([circuit], shots=1)
        result = job.result()
        alice_result = next(iter(result[0].data.alice.get_counts().keys()))
        alice_results.append(alice_result)
        eve_result = next(iter(result[0].data.eve.get_counts().keys()))
        eve_results.append(eve_result)
        bob_result = next(iter(result[0].data.bob.get_counts().keys()))
        bob_results.append(bob_result)
    return alice_bases, eve_bases, bob_bases, alice_results, eve_results, bob_results

n_bits = 20
e_bits = 10
alice_bases, eve_bases, bob_bases, alice_results, eve_results, bob_results = run(n_bits, e_bits)

output = "| # | Alice's basis | Bob's basis | Basis Match | Alice's results | Eve's basis | Eve's results | Bob's results | Eaves check |\n"
output += "|---|---------------|-------------|-------------|----------------|-------------|---------------|---------------|-------------|\n"
for i in range(n_bits):
    alice_basis = alice_bases[i]
    bob_basis = bob_bases[i]
    basis_match = "✓" if alice_basis == bob_basis else " "
    alice_result = alice_results[i]
    eve_basis = eve_bases[i]
    eve_result = eve_results[i]
    bob_result = bob_results[i]
    eaves_check = " " if alice_basis != bob_basis else "✓" if alice_result == bob_result else "✗"

    output += f"| {i+1} | {alice_basis} | {bob_basis} | {basis_match} | {alice_result} | {eve_basis} | {eve_result} | {bob_result} | {eaves_check} | \n"

display(Markdown(output))

| # | Alice's basis | Bob's basis | Basis Match | Alice's results | Eve's basis | Eve's results | Bob's results | Eaves check |
|---|---------------|-------------|-------------|----------------|-------------|---------------|---------------|-------------|
| 1 | DA | DA | ✓ | 0 | HV | 0 | 0 | ✓ | 
| 2 | HV | DA |   | 0 | HV | 0 | 1 |   | 
| 3 | HV | HV | ✓ | 0 | HV | 0 | 0 | ✓ | 
| 4 | DA | DA | ✓ | 0 | HV | 0 | 0 | ✓ | 
| 5 | DA | DA | ✓ | 0 | HV | 0 | 0 | ✓ | 
| 6 | DA | HV |   | 0 | DA | 0 | 1 |   | 
| 7 | HV | DA |   | 0 | DA | 1 | 0 |   | 
| 8 | DA | DA | ✓ | 0 | HV | 0 | 0 | ✓ | 
| 9 | HV | DA |   | 1 | HV | 1 | 0 |   | 
| 10 | DA | DA | ✓ | 0 | HV | 0 | 0 | ✓ | 
| 11 | HV | HV | ✓ | 0 | HV | 0 | 0 | ✓ | 
| 12 | DA | DA | ✓ | 0 | HV | 0 | 0 | ✓ | 
| 13 | HV | HV | ✓ | 1 | HV | 1 | 1 | ✓ | 
| 14 | HV | DA |   | 0 | DA | 1 | 1 |   | 
| 15 | HV | HV | ✓ | 1 | HV | 1 | 1 | ✓ | 
| 16 | DA | HV |   | 1 | DA | 1 | 1 |   | 
| 17 | HV | HV | ✓ | 0 | DA | 1 | 1 | ✗ | 
| 18 | DA | DA | ✓ | 0 | DA | 1 | 1 | ✗ | 
| 19 | HV | HV | ✓ | 0 | DA | 0 | 0 | ✓ | 
| 20 | DA | HV |   | 0 | HV | 0 | 1 |   | 
